In [25]:
import sys
import os
from pathlib import Path

import sys
import os
from pathlib import Path
parent_dir = os.path.dirname(os.getcwd())
sys.path.append(parent_dir)
sys.path.append("../models")
processed_series_path = Path(parent_dir) / 'processed_series'
import copy
from scipy.special import erf
from scipy.stats import norm
import numpy as np
import matplotlib.pyplot as plt
import pickle

print(sys.path)
from ICL import MultiResolutionPDF



['/home/admin-quad/Toni/LLMICL_inPCA/figures_analysis', '/home/admin-quad/anaconda3/envs/LLM/lib/python311.zip', '/home/admin-quad/anaconda3/envs/LLM/lib/python3.11', '/home/admin-quad/anaconda3/envs/LLM/lib/python3.11/lib-dynload', '', '/home/admin-quad/anaconda3/envs/LLM/lib/python3.11/site-packages', '/home/admin-quad/LLM/llama', '/home/admin-quad/Toni/LLMICL_inPCA', '../models', '/home/admin-quad/Toni/LLMICL_inPCA', '../models', '/home/admin-quad/Toni/LLMICL_inPCA', '../models']


In [26]:
all_files = sorted([path for path in processed_series_path.iterdir()], key=lambda x: x.name)
for index, file in enumerate(all_files):
    series_dict = pickle.load(file.open('rb'))
    llama_size = series_dict['llama_size']
    random_seed = series_dict['random_seed']
    kernel = series_dict.get('kernel', '')
    print(f"[{index}]: {file.name}, {llama_size}, {random_seed}, {kernel}")

[0]: uncorrelated_gaussian_centered_sigma_0.1_0.pkl, 13b, 2, 
[1]: uncorrelated_gaussian_centered_sigma_0.1_1.pkl, 13b, 3, 
[2]: uncorrelated_gaussian_centered_sigma_0.1_2.pkl, 7b, 2, 
[3]: uncorrelated_gaussian_centered_sigma_0.1_3.pkl, 7b, 3, 
[4]: uncorrelated_gaussian_centered_sigma_0.1_4.pkl, 70b, 2, 
[5]: uncorrelated_gaussian_centered_sigma_0.1_5.pkl, 70b, 3, 
[6]: uncorrelated_gaussian_centered_sigma_0.1_KDE_0.pkl, KDE, 2, gaussian
[7]: uncorrelated_gaussian_centered_sigma_0.1_KDE_1.pkl, KDE, 3, gaussian
[8]: uncorrelated_gaussian_centered_sigma_0.1_KDE_10.pkl, KDE, 2, cosine
[9]: uncorrelated_gaussian_centered_sigma_0.1_KDE_11.pkl, KDE, 3, cosine
[10]: uncorrelated_gaussian_centered_sigma_0.1_KDE_2.pkl, KDE, 2, epanechnikov
[11]: uncorrelated_gaussian_centered_sigma_0.1_KDE_3.pkl, KDE, 3, epanechnikov
[12]: uncorrelated_gaussian_centered_sigma_0.1_KDE_4.pkl, KDE, 2, tophat
[13]: uncorrelated_gaussian_centered_sigma_0.1_KDE_5.pkl, KDE, 3, tophat
[14]: uncorrelated_gaussian_cent

In [67]:

alpha = 1



file = all_files[24]
dist_type = 'uniform'

file = all_files[12]

file = all_files[22]

file = all_files[6]

file = all_files[4]

file = all_files[2]

file = all_files[16]

file = all_files[38]

file = all_files[12]

dist_type = 'gaussian'

file = all_files[54]
# file = all_files[56]
# file = all_files[57]
# file = all_files[59]
# file = all_files[54]
# file = all_files[52] 
# file = all_files[56] 

file = all_files[53]
file = all_files[55]
file = all_files[57]
file = all_files[59]
# file = all_files[58]
file = all_files[60]


file = all_files[24]
file = all_files[28]
file = all_files[30]
file = all_files[27]
dist_type = 'gaussian'


# file = all_files[54]
# file = all_files[55]
# file = all_files[56]
# file = all_files[57]

# file = all_files[65]
# file = all_files[66]
# file = all_files[67]
file = all_files[68]

# file = all_files[60]
# file = all_files[61]
file = all_files[63]
# file = all_files[64]

processed_dict = pickle.load(file.open('rb'))
full_series = processed_dict['full_series']
rescaled_true_mean_arr = processed_dict.get('rescaled_true_mean_arr',None)
rescaled_true_sigma_arr = processed_dict.get('rescaled_true_sigma_arr',None)
true_PDF = processed_dict.get('true_PDF',None)
llama_size = processed_dict['llama_size']
mode = processed_dict['mode']
refine_depth = processed_dict['refine_depth']
random_seed = processed_dict['random_seed']
PDF_list = processed_dict['PDF_list']
time_series = processed_dict['time_series']
prec = processed_dict['prec']
kernel = processed_dict.get('kernel', '')
print(f"seed: {processed_dict['random_seed']}")
print(f"model name: {llama_size}")
print(f"mode: {mode}")
print(f"refine_depth: {refine_depth}")
print(f"prec: {prec}")
print(f"kernel: {kernel}")
print(f"name: {file.stem}")

seed: 0
model name: KDE
mode: all
refine_depth: 1
prec: 2
kernel: epanechnikov
name: uncorrelated_random_PDF_l_0.1_KDE_3


In [68]:
### tune temperature 

for PDF in PDF_list:
    if alpha != 1:
        PDF.rescale_temperature(alpha)

In [69]:
### Calculate true discretized distribution: PDF_list_true
### Applies to gaussian distribution only

PDF_true_list = copy.deepcopy(PDF_list)
if true_PDF is not None:
    for i in range(len(PDF_true_list)):
        PDF_true_list[i] = true_PDF
else:
    for i, (PDF, PDF_true, true_mean, true_sigma) in enumerate(zip(PDF_list, PDF_true_list, rescaled_true_mean_arr, rescaled_true_sigma_arr)):
        def cdf(x):
            if dist_type == "gaussian":
                return 0.5 * (1 + erf((x - true_mean) / (true_sigma * np.sqrt(2))))
            elif dist_type == "uniform":
                lower_bound = true_mean - true_sigma
                upper_bound = true_mean + true_sigma
                return np.where(
                    (x >= lower_bound) & (x <= upper_bound),
                    (x - lower_bound) / (2 * true_sigma),
                    np.where(x < lower_bound, 0, 1)
                )
            else:
                raise NotImplementedError("This distribution type is not implemented.")
        
        PDF_true.discretize(cdf, mode="cdf")
        PDF_true.compute_stats()
        PDF_true_list[i] = PDF_true

In [70]:
from ipywidgets import interact
import matplotlib.pyplot as plt
from scipy.stats import norm, uniform
save_path = f"../figures/BM_example_temp{alpha}.png"
### Load multiple digits to MultiResolutionPDF
comma_locations = np.sort(np.where(np.array(list(full_series)) == ',')[0])
time_series_rescaled = (time_series-time_series.min()) / (time_series.max()-time_series.min()) * (8.5-1.5) + 1.5

plot1_log_scale = 0
log_scale = 0
truth_PDF = 1
# final_state = 709
final_state = None

### Plot distribution before ith comma
def digiprob_plotter(comma_idx=1):
    if comma_idx == 0:
        start_idx = 0
    else:
        start_idx = comma_locations[comma_idx-1]+1
    if truth_PDF:
        fig, axs = plt.subplots(3, 1, figsize=(8, 5), dpi = 200)
    else:
        fig, axs = plt.subplots(2, 1, figsize=(8, 10/3), dpi = 200)
    # Adjust the horizontal space between subplots
    plt.subplots_adjust(hspace=0.7)
    # Plot the full array with a marker on the selected value
    if final_state is not None:
        time_series_plot = time_series_rescaled[:final_state+1]
        axs[0].set_xlim(-10, final_state+10)
    else:
        time_series_plot = time_series_rescaled
    axs[0].plot(time_series_plot[:-1], marker='o', color='black', markersize=2, lw = '0.1', label = "observation")
    axs[0].scatter(comma_idx, time_series_plot[comma_idx], color='r', marker='o', label = "prediction")
    axs[0].set_xlabel('context lenght')
    axs[0].set_ylabel(r'rescaled $x$')
    
    if plot1_log_scale:
        axs[0].set_yscale('log')
        
    # Plot softmax distributions for each digit
    axs[1].set_ylabel("probability density")
    # axs[1].set_title(full_series[start_idx-30:start_idx] + "?")      
    PDF_list[comma_idx].compute_stats()
    PDF_list[comma_idx].plot(ax = axs[1], log_scale=log_scale, statistic = False)
    if truth_PDF:
        PDF_true_list[comma_idx].plot(ax = axs[2], log_scale=log_scale, statistic = False)
        axs[2].set_title('discretized truth')
        # axs[2].set_xlabel("Digit")
        axs[2].set_xlim(0,10)
    
    # # characterizing ground truth distribution
    # true_mean = rescaled_true_mean_arr[comma_idx]
    # true_sigma = rescaled_true_sigma_arr[comma_idx]
    # if true_sigma == 0:
    #     axs[1].vlines(true_mean, 0, np.max(PDF_list[comma_idx].bin_height_arr), color='r', label='Truth', lw = 3, alpha = 0.7)
    # else:
    #     x_values = np.linspace(0, 10, 300)
    #     if dist_type == "gaussian":
    #         true_PDF = norm.pdf(x_values, true_mean, true_sigma)
    #     elif dist_type == "uniform": 
    #         true_PDF = uniform.pdf(x_values, true_mean-true_sigma, true_sigma*2)
    #     else:
    #         raise NotImplementedError("This distribution type is not implemented.")
    #     axs[1].plot(x_values, true_PDF, color = 'r', label = 'Truth', lw = 3, alpha = 0.7)
    # axs[1].set_xlabel(r'rescaled $x$')
    # axs[1].bar([0], [0], color='dodgerblue', label='Prediction')
    axs[1].set_xlim(0,10)
    # axs[1].set_ylim(0,5)
    axs[1].set_title('estimated density')
    axs[0].legend()
    axs[1].legend()
    if log_scale:
        axs[1].set_ylim(0.005,10)
        if truth_PDF:
            axs[2].set_ylim(0.0005,10)
    if final_state is not None: 
        plt.tight_layout()
        plt.savefig(save_path)

# Adjust the range of the slider to match the number of commas in the series
num_commas = full_series.count(',')
if final_state is not None: 
    interact(digiprob_plotter, comma_idx=final_state)
else:
    interact(digiprob_plotter, comma_idx=(1, num_commas-1, 1))

interactive(children=(IntSlider(value=1, description='comma_idx', max=399, min=1), Output()), _dom_classes=('w…